# Script to evaluate the ML-Models

Setup (load automl-models.pickle)

In [ ]:
import pickle
import csv

with open(r"../app/_meta.csv") as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')
  list_qualities = list()
  for line in csv_reader:
    list_qualities.append(line[0])
  list_qualities = list_qualities[1:]


with open(r"../app/ml-models.pickle", "rb") as input_file:
  ml_models = pickle.load(input_file)

Inspect the score:

In [ ]:
sum_scores = 0
for key, value in ml_models.items():
  print(key, ':', value.get('score'))
  sum_scores += value.get('score')
print()
print('Average:', "{:.2f}".format(100*sum_scores/len(ml_models), 2), '%')

Test with a random dataset:

In [ ]:
import openml as oml
import pandas as pd

In [ ]:
dataset = oml.datasets.get_dataset(12)

In [ ]:
t = dict()

for quality in list_qualities:
  t[quality] = [dataset.qualities.get(quality)]
test_df = pd.DataFrame(t)

for key, value in ml_models.items():
  aml = value.get('model')
  print(key, ':', aml.predict(test_df))

In [ ]:
for key, value in ml_models.items():
  aml = value.get('model')
  print(key, ':', aml.best_estimator)

In [ ]:
from sklearn.inspection import permutation_importance
import numpy as np
import pandas as pd

results = dict()

for key in ml_models.keys():
  ml_model_dict = ml_models.get(key)
  model, X, y = ml_model_dict.get('model'), ml_model_dict.get('X'), ml_model_dict.get('y')
  np.nan_to_num(X)
  result = permutation_importance(model,
                                X,
                                y,
                                n_repeats=10,
                                random_state=42,
                                n_jobs=-1)
  results[key] = result

In [ ]:
import matplotlib.pyplot as plt

for key, result in results.items(): 
  sorted_idx = result.importances_mean.argsort()[-5:]      
  fig, ax = plt.subplots()
  ax.boxplot(result.importances[sorted_idx].T,
           vert=False,
           labels=X.columns[sorted_idx])
  fig.tight_layout()
  plt.savefig('graphs/' + key + '-features.png', dpi=300)